# Add Titles of Works and Work IDs to Database

Now I need to add the titles of works and their work IDs to the mix. I have those in a file downloaded from the DLL Catalog. The Greek authors will not have any works associated with their names since I don't need them.

In [1]:
import pandas as pd

In [2]:
# Read in the Latin and Greek CSV
latin_greek = pd.read_csv('fresh/latin_greek.csv')
# Make a dataframe of the Works and Work IDs
works = pd.read_csv('data/dll-works-and-authors.csv')

In [3]:
# Get information about the two dataframes
print(latin_greek.info())
print(works.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5515 entries, 0 to 5514
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   DLL ID           5515 non-null   object
 1   Authorized Name  5515 non-null   object
 2   Variant Names    5515 non-null   object
 3   Latin            5515 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 172.5+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5279 entries, 0 to 5278
Data columns (total 9 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   Title                        5279 non-null   object
 1   Work Name                    5279 non-null   object
 2   Alternative Title            484 non-null    object
 3   Short Title                  4 non-null      object
 4   Author Name Latin            4753 non-null   object
 5   Author Name Native Language  2941 non-null   object
 6   

In [4]:
# Synchronize the column names
latin_greek = latin_greek.rename(columns={'DLL ID': 'DLL Identifier (Author)',})

In [5]:
# Drop some of the unnecessary columns in works
works = works [['DLL Identifier (Author)','DLL Identifier (Work)','Title','Alternative Title']]

Right now, there is a single row for each individual work. That means that an author's ID can appear multiple times in the `works` data, but only once in the `database` data. I need to combine the rows in works so that there is a single row for each author.

In [6]:
# Combine the Title and Alternative Title into "Title (Alternative TItle)"
works["Combined Titles"] = works.apply(
    lambda row: f"{row['Title']} ({row['Alternative Title']})" if pd.notna(row["Alternative Title"]) and row["Alternative Title"] 
    else row["Title"] if row["Title"] 
    else "",
    axis=1
)

The following block makes a new dataframe called `grouped` by calling `groupby()` on the `DLL Identifier (Author)` column of `works`, then applying a `lambda` function to the `DLL Identifier (Work)` and `Combined Titles` columns. If there's more than one work, they will be separated by a semicolon and a space. Otherwise, there will just be a work ID and the combined titles. If there aren't any works, then "No works available" will be added.

In [7]:
grouped = works.groupby('DLL Identifier (Author)')[['DLL Identifier (Work)','Combined Titles']].apply(
    lambda group: (
        ";".join(
            f"{row['DLL Identifier (Work)']}: {row['Combined Titles']}" for _, row in group.iterrows()
        )
        if len(group) > 1  # Check if there is more than one work
        else f"{group.iloc[0]['DLL Identifier (Work)']}: {group.iloc[0]['Combined Titles']}"  # Single work
        if len(group) == 1
        else "No works available"
    )
).reset_index().rename(columns={0:'Combined Works'})

In [8]:
display(grouped)

,DLL Identifier (Author),Combined Works
0,A2491,W5039: In Decem Libros Priores Elementorum Euc...
1,A2868,W3325: Fragmenta
2,A2869,W1662: Lupercalia
3,A2870,W5168: Iurisprudentia
4,A2871,W1628: Epist. ab M. Claudio Marcello
...,...,...
1384,A5727,W1967: Fragmenta
1385,A5730,W453: Fragmenta
1386,A5986,W10631: De facultatibus alimentorum ex Paulo A...
1387,A907,W499: Grammatica;W560: Iurisprudentia


Now I need to merge the `database` and `grouped` dataframes to bring the `Combined Works` into the mix. I'm using a `how='left'` to get the "Cartesian product" of the two dataframes. That is, I should end up with a new dataframe that has the same number of rows as `database`, but with `Combined Works` added to the rows with matching `DLL Identifier (Author)` values.

In [9]:
# Merge the dataframes so that there's only one
# Use a left join to preserve the Cartesian product
merged = pd.merge(latin_greek,grouped,on='DLL Identifier (Author)',how='left')

In [11]:
# Compare the number of rows. They should be the same.
print(len(latin_greek))
print(len(merged))

5515
5515


In [12]:
display(merged)

,DLL Identifier (Author),Authorized Name,Variant Names,Latin,Combined Works
0,A1868,"Herryson, Joannes","Herryson, Joannes floruit=15th Century A.D. | ...",1,NaN
1,A1870,"Stratford, John, -1348",Johannes Stratford | John Stratford | John Str...,1,NaN
2,A2181,"Nicomachus, of Gerasa","Nicòmac, de Gerasa, actiu segle I | Nicomachus...",1,NaN
3,A2491,"Anaritius, -approximately 922",Abū al-ʿAbbās al-Faḍl ibn Ḥātim Nayrīzī...,1,W5039: In Decem Libros Priores Elementorum Euc...
4,A2492,"Strecker, Karl, 1861-1945","Karl Strecker | Strecker, Karl | Strecker, Kar...",1,NaN
...,...,...,...,...,...
5510,G73664,"Leo, VI, Emperor of the East 866-912",Léon VI | 0866-0912 | empereur d'Orient | L...,0,NaN
5511,G54624,"Xenocrates, of Chalcedon, approximately 396 B....",Xenocrates | of Chalcedon | approximately 39...,0,NaN
5512,G28402,"Thaletas, Musicus, um. 665 v. chr.",Taletes | Thaletas Musicus v665,0,NaN
5513,G29158,"Pachymérès, George, 1242-approximately 1310",Pachymère | Georges 1242-1310? | Pachymeres ...,0,NaN


In the previous steps, I thought it was a good idea to use the pipe character as a delimiter when combining values. Since I need to use that character to delineate all the columns when I combine them into the "Text" column, I should replace the pipe with a semicolon.

In [13]:
# Replace pipe with semicolon in columns
merged['Variant Names'] = merged['Variant Names'].str.replace(" | ",";")
display(merged['Variant Names'])

0       Herryson, Joannes floruit=15th Century A.D.;Jo...
1       Johannes Stratford;John Stratford;John Stratfo...
2       Nicòmac, de Gerasa, actiu segle I;Nicomachus, ...
3       Abū al-ʿAbbās al-Faḍl ibn Ḥātim Nayrīzī...
4       Karl Strecker;Strecker, Karl;Strecker, Karl, 1...
                              ...                        
5510    Léon VI; 0866-0912; empereur d'Orient; Leo VI....
5511    Xenocrates; of Chalcedon; approximately 396 B....
5512                       Taletes; Thaletas Musicus v665
5513    Pachymère; Georges 1242-1310?; Pachymeres; Geo...
5514    Philoxène de Cythère 0435?-0380?; Philoxenus; ...
Name: Variant Names, Length: 5515, dtype: object

In [14]:
# Check the columns
merged.columns

Index(['DLL Identifier (Author)', 'Authorized Name', 'Variant Names', 'Latin',
       'Combined Works'],
      dtype='object')

I was also being too clever by half in using 0 and 1 to label authors as Greek or Latin, respectively. I'll use the `map()` method to take care of that.

In [15]:
# Change the values of the Latin column to "Latin" (1) and "Greek" (0)
latin_dict = {1:'Latin',0:'Greek'}
merged['Latin'] = merged['Latin'].map(latin_dict).astype("str")

I need to combine the columns, except `Text` (which was a mistake—I made it too soon in a previous step), into one column, with the different values separated by a pipe. That's a relatively simple operation.

In [17]:
# Replace NaN with an empty string before concatenation
merged['Text'] = (
    merged['DLL Identifier (Author)'].fillna('') + '|' +
    merged['Authorized Name'].fillna('') + '|' +
    merged['Variant Names'].fillna('') + '|' +
    merged['Latin'].fillna('').astype('str') + '|' +
    merged['Combined Works'].fillna('')
)

In [18]:
display(merged)

,DLL Identifier (Author),Authorized Name,Variant Names,Latin,Combined Works,Text
0,A1868,"Herryson, Joannes","Herryson, Joannes floruit=15th Century A.D.;Jo...",Latin,NaN,"A1868|Herryson, Joannes|Herryson, Joannes flor..."
1,A1870,"Stratford, John, -1348",Johannes Stratford;John Stratford;John Stratfo...,Latin,NaN,"A1870|Stratford, John, -1348|Johannes Stratfor..."
2,A2181,"Nicomachus, of Gerasa","Nicòmac, de Gerasa, actiu segle I;Nicomachus, ...",Latin,NaN,"A2181|Nicomachus, of Gerasa|Nicòmac, de Gerasa..."
3,A2491,"Anaritius, -approximately 922",Abū al-ʿAbbās al-Faḍl ibn Ḥātim Nayrīzī...,Latin,W5039: In Decem Libros Priores Elementorum Euc...,"A2491|Anaritius, -approximately 922|Abū al-ʿA..."
4,A2492,"Strecker, Karl, 1861-1945","Karl Strecker;Strecker, Karl;Strecker, Karl, 1...",Latin,NaN,"A2492|Strecker, Karl, 1861-1945|Karl Strecker;..."
...,...,...,...,...,...,...
5510,G73664,"Leo, VI, Emperor of the East 866-912",Léon VI; 0866-0912; empereur d'Orient; Leo VI....,Greek,NaN,"G73664|Leo, VI, Emperor of the East 866-912|Lé..."
5511,G54624,"Xenocrates, of Chalcedon, approximately 396 B....",Xenocrates; of Chalcedon; approximately 396 B....,Greek,NaN,"G54624|Xenocrates, of Chalcedon, approximately..."
5512,G28402,"Thaletas, Musicus, um. 665 v. chr.",Taletes; Thaletas Musicus v665,Greek,NaN,"G28402|Thaletas, Musicus, um. 665 v. chr.|Tale..."
5513,G29158,"Pachymérès, George, 1242-approximately 1310",Pachymère; Georges 1242-1310?; Pachymeres; Geo...,Greek,NaN,"G29158|Pachymérès, George, 1242-approximately ..."


In [19]:
# Save to a CSV
merged.to_csv('fresh/all_data.csv',index=False)